## Continuous - Temporal

Which time field(s) are useful and pertain to other measurement fields? The User Guide does not mention anlys_time, while the GLIMS Description of fields describes anlys_time as when analysis was carried out. Both associate src_date with the glacier outlines. Exploration is inconclusive.

According to the [Summary Statistics](http://glims.colorado.edu/glacierdata/db_summary_stats.php) page, 173007 out of 444633 glaciers have multitemporal coverage.

In [1]:
polygons2.dtypes

NameError: name 'polygons2' is not defined

In [ ]:
polygons2.iloc[:,[3,12]].sample(10)

In [ ]:
polygons2.iloc[:,[3,12]].describe()

In [ ]:
# src_date is usually older than anlys_time
polygons2.src_date.hist(label='src_date');
polygons2.anlys_time.hist(label='anlys_time');
plt.title('anlys_time');
plt.legend();

(polygons2['anlys_time'] > polygons2['src_date']).value_counts()

## Continuous - Area

Area of glacier coverage (latest snapshot), km2	1.20007e+06
Note: A good working number for area is approximately 750,000 km2 (not including the ice sheets of Greenland and Antarctica).
Earliest analysis	1750-01-01
Most recent analysis	2021-02-26
Number of glaciers with multi-temporal coverage	173148






db_area, calculated by the NSIDC, seems quite complete. 

area may be less complete, and width and length seem highly incomplete.


In [ ]:
dimensions = polygons2.iloc[:, 3:7]
dimensions

In [ ]:
dimensions.hist();
plt.tight_layout();

In [ ]:
# How can there be negative min values?
dimensions.describe()

In [ ]:
# their scales are quite different
dimensions.boxplot(flierprops={'marker': 'o', 'markersize': 5, 'markerfacecolor': 'red'});

In [ ]:
# most of the data is around zero
fig, axs = plt.subplots(2, 4)
axs[0, 0].boxplot(dimensions.area, flierprops={'marker': 'o', 'markersize': 5,  'markerfacecolor': 'red'});
axs[0, 1].violinplot(dimensions.area);
axs[0, 2].boxplot(dimensions.db_area, flierprops={'marker': 'o', 'markersize': 5, 'markerfacecolor': 'red'});
axs[0, 3].violinplot(dimensions.db_area);
axs[1, 0].boxplot(dimensions.width, flierprops={'marker': 'o', 'markersize': 5, 'markerfacecolor': 'red'});
axs[1, 1].violinplot(dimensions.width);
axs[1, 2].boxplot(dimensions.length, flierprops={'marker': 'o', 'markersize': 5, 'markerfacecolor': 'red'});
axs[1, 3].violinplot(dimensions.length);

In [ ]:
# width and length are > 90% zero - because of lack of data?
# area also has substantial zeros, while db_area has hardly any zeroes
# db_area is more complete
dimensions.eq(0).sum() / dimensions.shape[0]

In [ ]:
# Proceed with db_area
dimensions.db_area.describe()

In [ ]:
# The ones with a negative area
plt.violinplot(dimensions[dimensions.db_area < 0].db_area);

In [ ]:
plt.violinplot(dimensions[dimensions.db_area > 0].db_area);

In [ ]:
pos = dimensions[dimensions.db_area > 0].db_area
plt.violinplot(pos[pos<1]);


In [ ]:
np.log(dimensions.db_area + 1).hist();

## Continuous - Elevation
Half of glaciers have elevations around sealevel, and a few below, which seems dubious as they are usually high in the mountains.

In [ ]:
elevations = polygons2.iloc[:, 8:11]
elevations

In [ ]:
elevations.hist();
plt.tight_layout();

In [ ]:
# Elevations have many zero values and a few negative values...?
plt.style.use('default')
plt.hist(elevations.min_elev, alpha=0.5, bins=30, label='Min Elevation')
plt.hist(elevations.mean_elev, alpha=0.5, bins=30, label='Mean Elevation')
plt.hist(elevations.max_elev, alpha=0.5, bins=30, label='Max Elevation')
plt.legend();
plt.title('Elevations');

In [ ]:
elevations.describe()

In [ ]:
# Quite a lot of data near zero (glaciers at sea level)
plt.violinplot(elevations);

In [ ]:
# Nearly half or more are zero
elevations.eq(0).sum() / elevations.shape[0]

In [ ]:
# Nearly half of the zeros coincide --- missing data, or at sea-level?
elevations[elevations.min_elev == elevations.max_elev].eq(0).sum() / elevations.shape[0]

In [ ]:
polygons2[polygons2.max_elev == polygons2.min_elev]

In [ ]:
# Negative values only seem to be around -10000 ...?
elevations.boxplot(flierprops={'marker': 'o', 'markersize': 5, 'markerfacecolor': 'red'});

In [ ]:
# These few glaciers are below sealevel
print(len(polygons2[polygons2.min_elev<0]))
print(len(polygons2[polygons2.mean_elev<0]))
print(len(polygons2[polygons2.max_elev<0]))

In [ ]:
# Most of them coincide
print(polygons2[ (polygons2.min_elev<0) & (polygons2.mean_elev<0) & (polygons2.max_elev<0)].shape)
print(polygons2[ (polygons2.min_elev<0) | (polygons2.mean_elev<0) | (polygons2.max_elev<0)].shape)

In [ ]:
# Where are they? Create of subset of these low glaciers for visualization
submarine = polygons2[ (polygons2.min_elev<0) | (polygons2.mean_elev<0) | (polygons2.max_elev<0)]
submarine.shape, submarine.crs 

In [ ]:
# They seem to be in NW USA and Nepal. Data errors?
fig, ax = plt.subplots(figsize=(10, 8))
world.boundary.plot(ax=ax, edgecolor = 'black', linewidth=0.2);
submarine['geometry'].centroid.plot(ax=ax, marker='.', color='red', markersize=10);
ax.set_xlim(submarine.total_bounds[0]-5, submarine.total_bounds[2]+5)
ax.set_ylim(submarine.total_bounds[1]-5, submarine.total_bounds[3]+5)